# ConnectWise PSA to Microsoft Fabric Integration

This notebook demonstrates how to use the fabric_api package to extract data from ConnectWise PSA and load it into Microsoft Fabric.

In [ ]:
# First cell - Install the package
%pip install /lakehouse/default/Files/dist/fabric_api-0.2.2-py3-none-any.whl

In [ ]:
# Configure environment and logging
import os
import logging
from datetime import datetime, timedelta

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

# Set credentials (replace with your actual credentials)
os.environ["CW_AUTH_USERNAME"] = "thekking+yemGyHDPdJ1hpuqx"  # Replace with your username
os.environ["CW_AUTH_PASSWORD"] = "yMqpe26Jcu55FbQk"  # Replace with your password
os.environ["CW_CLIENTID"] = "c7ea92d2-eaf5-4bfb-a09c-58d7f9dd7b81"  # Replace with your client ID

# Optional: Set other environment variables
os.environ["FABRIC_STORAGE_ACCOUNT"] = "onelake"

In [ ]:
# Initialize PySpark session
from pyspark.sql import SparkSession

# Get active Spark session in Fabric
spark = SparkSession.getActiveSession() or SparkSession.builder.getOrCreate()

# Display Spark version for verification
print(f"Using Spark version: {spark.version}")

In [ ]:
# Import the package - test if imports work correctly
from fabric_api.client import ConnectWiseClient
from fabric_api.connectwise_models import (
    Agreement,
    TimeEntry,
    ExpenseEntry,
    Invoice,
    UnpostedInvoice,
    ProductItem
)

# Test client connection
client = ConnectWiseClient()

# Try a simple API call to check connectivity
response = client.get("/system/info")
print(f"API connection status: {response.status_code}")
print("API response:")
import json
print(json.dumps(response.json(), indent=2))

In [ ]:
# Option 1: Run daily ETL with default settings
from fabric_api.pipeline import run_daily_etl

# Define lakehouse path
lakehouse_root = "/lakehouse/default/Tables/connectwise"

# Run ETL for last 30 days (default)
print("Running daily ETL process with default settings (last 30 days)...")
result = run_daily_etl(
    lakehouse_root=lakehouse_root,  # Where to store the data
    max_pages=1,  # Limit to 1 page for testing
    mode="overwrite"  # Use 'overwrite' for testing, 'append' for production
)

# Display results
print("\nDaily ETL Results:")
for entity_name, (record_count, error_count) in result.items():
    print(f"{entity_name}: {record_count} records, {error_count} errors")

In [ ]:
# Option 2: Process specific entities
from fabric_api.pipeline import process_entity

# Define parameters
entity_name = "TimeEntry"  # Choose from: Agreement, TimeEntry, ExpenseEntry, Invoice, UnpostedInvoice, ProductItem
start_date = (datetime.now() - timedelta(days=7)).strftime("%Y-%m-%d")
end_date = datetime.now().strftime("%Y-%m-%d")
table_path = f"{lakehouse_root}/{entity_name.lower()}"

print(f"Processing {entity_name} from {start_date} to {end_date}...")

# Process the entity
result_df, errors = process_entity(
    entity_name=entity_name,
    table_path=table_path,
    spark=spark,
    start_date=start_date,
    end_date=end_date,
    page_size=100,
    max_pages=1,  # Limit for testing
    write_mode="overwrite"  # Use 'overwrite' for testing, 'append' for production
)

# Show results
print(f"Processed {result_df.count()} {entity_name} records with {len(errors)} errors")
if result_df.count() > 0:
    print("\nSample data:")
    result_df.limit(5).show()

if errors:
    print(f"\nSample errors (showing first 3):")
    for error in errors[:3]:
        print(f"- {error.error_type}: {error.error_message}")

In [ ]:
# Option 3: Low-level API using extract functions
from fabric_api.extract.time import fetch_time_entries_raw
from fabric_api.connectwise_models import TimeEntry

# Fetch raw time entries (using client we already created)
print("Fetching raw time entries...")
raw_entries = fetch_time_entries_raw(
    client=client,
    page_size=10,
    max_pages=1,  # Limit for testing
    conditions=f"dateStart >= [{start_date}] and dateStart <= [{end_date}]"
)

print(f"Fetched {len(raw_entries)} raw time entries")

# Validate using models
valid_entries = []
validation_errors = []

for entry in raw_entries:
    try:
        # Validate using Pydantic model
        validated = TimeEntry.model_validate(entry)
        valid_entries.append(validated)
    except Exception as e:
        validation_errors.append({"id": entry.get("id"), "error": str(e)})
        
print(f"Validated {len(valid_entries)} entries with {len(validation_errors)} errors")

# Display first entry if available
if valid_entries:
    print("\nSample validated entry:")
    print(valid_entries[0].model_dump_json(indent=2))

In [ ]:
# Verify data was written to Delta tables
from fabric_api.storage.delta import read_delta_table, table_exists

# Check the tables that were created
entity_types = ["agreement", "timeentry", "expenseentry", "invoice", "unpostedinvoice", "productitem"]

print("Checking Delta tables:")
for entity in entity_types:
    path = f"{lakehouse_root}/{entity}"
    exists = table_exists(spark, path)
    print(f"- {entity}: {'✅ Exists' if exists else '❌ Not found'}") 
    
    if exists:
        # Read the table
        df = read_delta_table(spark, path)
        count = df.count()
        print(f"  Records: {count}")
        
        # Show schema
        print("  Schema:")
        df.printSchema()